# Buffer Overflow

- one of the top 25 most dangerous software errors according to Common Weakness Enemuration (CWE) for several years as recently as 2019 [https://cwe.mitre.org/top25/archive/2019/2019_cwe_top25.html](https://cwe.mitre.org/top25/archive/2019/2019_cwe_top25.html)
- most Internet worms use buffer overflow vulnerabilities to propagate
- C and C++ assume that programmers are responsible for data integrity
- modern compilers and OSes provide various protections making it harder to exploit traditional buffer overflows
- once the buffer/memory is allocated, there's no built-in mechanism to safeguard the amount of data that can be stored into
    - this is called buffer overrun or buffer overflow
    - user data may overwrite other part of the memory it's not supposed to making the program behave differently or crashing it
    - attackers could take advantage of this flaw to control the program and may pawn the system by executing shellcode or payload of their choice
- buffer overflow can violate all three core principles of **CIA** triad - **Confidentiality, Integrity, and Availability**
- **Confidentiality**: allows attackers to read confidential or data that they're not supposed to have access to
- **Integrity**: allows attackers to modify part of the data/program that they're not supposed to
- **Availability**: allows attackes to crash the system (SEGFAULT) violating availbility of the service

- typically two types:
    1. stack overflow
    2. heap overflow
- in this chapter, we'll focus on stack overflow

## Disable ASLR and other compiler protections
- compile.sh script disables ASLR and compiler protections

In [1]:
# if value is 0; ASLR is disabled
! cat /proc/sys/kernel/randomize_va_space

0


In [ ]:
# run this on terminal
# can't interactively type sudo password!
! echo 0 | sudo tee /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 

## Program's memory and stack may shift
- programs and data must be loaded into RAM inorder to execute
- depending on how the program is launched, several factors make the program's location in memory shift
    - even after virtual-address space randomization is disabled
- e.g. environment variables, length of program name, argument length to program, etc.

### see the difference between stack_overflow_demo.cpp and stack_overflow_demo1.cpp
- stack_overflow_demo.cpp doesn't take any argument from commandline
    - interactive program that prompts user to enter some text
- stack_overflow_demo1.cpp takes
    - uses argument provided to the program
    - doesn't prompt user
- let's compile and run these programs to see the address of buffer

In [153]:
%%bash
cp ./booksrc/stack_overflow_demo.cpp .
cp ./booksrc/stack_overflow_demo1.cpp .

In [154]:
! cat stack_overflow_demo.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

void bad() {
    char buffer[100];
    printf("buffer is at %p\n", buffer);
    cout << "Enter some text: ";
    cin >> buffer;
    cout << "buffer contains: \n" << buffer << endl;
}

int main(void) {
    bad();
    cout << "Good bye!" << endl;
    return 0;
}

In [155]:
! cat stack_overflow_demo1.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

void bad(char *data) {
    char buffer[100];
    printf("buffer is at %p\n", buffer);
    strcpy(buffer, data);
    cout << "buffer contains: \n" << buffer << endl;
}

int main(int argc, char *argv[]) {
    bad(argv[1]);
    cout << "Good bye!" << endl;
    return 0;
}

### buffer address in stack_overflow_demo.cpp

In [156]:
# let's compile the program and run it
! echo kali | sudo -S ./compile.sh stack_overflow_demo.cpp stack_overflow_demo.exe

[sudo] password for kali: 

In [157]:
# since stack_overflow_demo.exe is an interactive program,
# you must pipe some data into the program on Jupyter Notebook
# On terminal, you can either pipe or interactively type when prompted
! echo "Hello_World" | ./stack_overflow_demo.exe
# Note the address of buffer!

buffer is at 0xffffc2dc
Enter some text: buffer contains: 
Hello_World
Good bye!


In [158]:
! python -c 'print("A"*100)' | ./stack_overflow_demo.exe
# Note the address of buffer!

buffer is at 0xffffc2dc
Enter some text: buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [159]:
! python -c 'print("A"*200)' | ./stack_overflow_demo.exe
# Note the address of buffer!

buffer is at 0xffffc2dc
Enter some text: buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
/bin/bash: line 1: 39852 Done                    python -c 'print("A"*200)'
     39853 Segmentation fault      | ./stack_overflow_demo.exe


- Notice the same address for buffer in stack_overflow_demo.exe no matter how long the user-entered text is

## buffer address in stack_overflow_demo1.cpp
- compile and run the program and notice the address of buffer at different location

In [160]:
! echo kali | sudo -S ./compile.sh stack_overflow_demo1.cpp stack_overflow_demo1.exe

[sudo] password for kali: 

In [161]:
# run the program with an argument
# Note you CAN'T pipe the argument into the program as in previous example!
! ./stack_overflow_demo1.exe "Hello World"
# Note the address of buffer

buffer is at 0xffffc2bc
buffer contains: 
Hello World
Good bye!


In [162]:
! ./stack_overflow_demo1.exe "Hello World"
# Note the address of buffer; no change because same program is run with the same length of argument

buffer is at 0xffffc2bc
buffer contains: 
Hello World
Good bye!


In [163]:
! ./stack_overflow_demo1.exe $(python -c 'print("A"*100)')
# Note the address of buffer!

buffer is at 0xffffc26c
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [164]:
! ./stack_overflow_demo1.exe $(python -c 'print("A"*200)')
# Note the address of buffer!

buffer is at 0xffffc20c
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


## length of the program name
- let's give names of different lengths to the same program and notice the address of buffer
- this is true regardless the program takes an argument or not 
    - NOTE the name of the program itself is passed as an argument to the program as an environment variable

In [165]:
# let's compile the program and run stack_overflow_demo.cpp
! echo kali | sudo -S ./compile.sh stack_overflow_demo.cpp stack_overflow_demo.exe

[sudo] password for kali: 

In [166]:
! echo "Hello_World" | ./stack_overflow_demo.exe

buffer is at 0xffffc2dc
Enter some text: buffer contains: 
Hello_World
Good bye!


In [167]:
# if the same program is run multiple times, 
! echo "Hello_World" | ./stack_overflow_demo.exe
# Notice no change in buffer address

buffer is at 0xffffc2dc
Enter some text: buffer contains: 
Hello_World
Good bye!


In [168]:
# let's compile the program and run stack_overflow_demo.cpp
! echo kali | sudo -S ./compile.sh stack_overflow_demo.cpp stack.exe

[sudo] password for kali: 

In [169]:
! echo "Hello_World" | ./stack.exe

buffer is at 0xffffc30c
Enter some text: buffer contains: 
Hello_World
Good bye!


In [170]:
# let's compile the program and run stack_overflow_demo.cpp; output program name is stack1.exe
! echo kali | sudo -S ./compile.sh stack_overflow_demo.cpp stack1.exe

[sudo] password for kali: 

In [171]:
! echo "Hello_World" | ./stack1.exe

buffer is at 0xffffc30c
Enter some text: buffer contains: 
Hello_World
Good bye!


In [172]:
# let's compile the program and run stack_overflow_demo.cpp; output program name is stack11.exe
! echo kali | sudo -S ./compile.sh stack_overflow_demo.cpp stack11.exe

[sudo] password for kali: 

In [173]:
! echo "Hello_World" | ./stack11.exe
# perhaps, filename is not long engouh to make a difference

buffer is at 0xffffc30c
Enter some text: buffer contains: 
Hello_World
Good bye!


In [174]:
%%bash
# let's compile the program and run stack_overflow_demo.cpp; output program name is stack11111.exe
echo kali | sudo -S ./compile.sh stack_overflow_demo.cpp stack11111.exe
echo "Hello_World" | ./stack11111.exe

buffer is at 0xffffc2fc
Enter some text: buffer contains: 
Hello_World
Good bye!


[sudo] password for kali: 